# NB_170124T1426_logistic_regression_pytorch

# 1.Goal

- implementation logistic regression with pytorch

# 2.Imports

In [43]:
import numpy as np
import torch
import torch.nn as nn
from typing import Union
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 3.Implemetation of the class logistic regression

In [21]:
class ModelLogisticRegression(nn.Module):
    def __init__(self, in_features, out_features: int = 1):
        super(ModelLogisticRegression, self).__init__()
        self.linear = nn.Linear(
            in_features=in_features, out_features=out_features
        )

    def forward(self, X):
        Y_hat = torch.sigmoid(self.linear(X))
        return Y_hat

In [114]:
class LogisticRegressionPytorch(object):
    def __init__(self, learning_rate: float = 0.01, num_epochs: int = 100):
        self.lr = learning_rate
        self.num_epochs = num_epochs
        self.model = None
        self.optimizer = None
        self.standard_scalar = None

    def initialize_model(self, in_features, out_features: int = 1):
        self.model = ModelLogisticRegression(
            in_features=in_features, out_features=out_features
        )

    def initialize_optim(self):
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr)

    def initialize_loss_function(self):
        self.loss_function = nn.BCELoss()

    def initialize_standard_scalar(self):
        self.standard_scalar = StandardScaler()

    def fit(
        self,
        X: Union[np.ndarray, torch.Tensor],
        Y: Union[np.ndarray, torch.Tensor],
    ):
        # make scaling
        self.initialize_standard_scalar()
        X = self.standard_scalar.fit_transform(X)

        # convert into torch.Tensor
        if isinstance(X, np.ndarray):
            X = torch.from_numpy(X.astype(np.float32))

        if isinstance(Y, np.ndarray):
            Y = torch.from_numpy(Y.astype(np.float32))

        Y = Y.view(-1, 1)

        _, in_features = X.size()

        # initializing model, loss_function and optimizer
        self.initialize_model(in_features=in_features)
        self.initialize_loss_function()
        self.initialize_optim()

        print(f"{Y=}")

        for epoch in range(self.num_epochs):
            # forward pass
            Y_hat = self.model(X)

            # compute loss
            loss = self.loss_function(Y_hat, Y)

            # backward pass
            loss.backward()

            # update model parameters
            self.optimizer.step()

            # zero gradients
            self.optimizer.zero_grad()

            # print the loss every 10 epochs
            if epoch % 10 == 0:
                print(f"{epoch=}, {loss.item():.4f}")

    def predict(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.standard_scalar.transform(X)
        X_scaled_tensor = torch.from_numpy(X_scaled.astype(np.float32))
        with torch.no_grad():
            Y_hat = self.model(X_scaled_tensor)

        Y_hat_np = Y_hat.numpy()

        return Y_hat_np

    def predict_cls(self, X: np.ndarray) -> np.ndarray:
        Y_hat = self.predict(X)
        Y_hat_cls = np.where(Y_hat >= 0.5, 1, 0)

        return Y_hat_cls

    def eveluate_model(self, X: np.ndarray, Y: np.ndarray):
        Y_hat_cls = self.predict_cls(X)
        accuracy_result = np.sum([Y_hat_cls.ravel() == Y]) / Y.shape[0]

        return accuracy_result

# 3.Load data

In [99]:
bd = datasets.load_breast_cancer()
X, Y = bd["data"], bd["target"]

In [110]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# 4.Use model

- initialize the model

In [115]:
regressor = LogisticRegressionPytorch()

- fit the model

In [116]:
regressor.fit(X_train, Y_train)

Y=tensor([[0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
    

- eveluate the model

In [117]:
regressor.eveluate_model(X_test, Y_test)

0.9649122807017544